In [1]:
import pandas as pd 
import numpy as np 
import cv2

In [2]:
train = pd.read_csv("train.txt", sep=" ", header=None)
test = pd.read_csv("test.txt", sep=" ", header = None)

In [3]:
train = train.rename(columns = {0:"image", 1:"class"})
train

,image,class
0,TIN/n01443537/images/n01443537_0.JPEG,0
1,TIN/n01443537/images/n01443537_1.JPEG,0
2,TIN/n01443537/images/n01443537_10.JPEG,0
3,TIN/n01443537/images/n01443537_100.JPEG,0
4,TIN/n01443537/images/n01443537_101.JPEG,0
...,...,...
99595,TIN/n12267677/images/n12267677_93.JPEG,199
99596,TIN/n12267677/images/n12267677_94.JPEG,199
99597,TIN/n12267677/images/n12267677_95.JPEG,199
99598,TIN/n12267677/images/n12267677_96.JPEG,199


In [4]:
test = test.rename(columns = {0:"image", 1:"class"})
test

,image,class
0,TIN/n01443537/images/n01443537_99.JPEG,0
1,TIN/n01629819/images/n01629819_99.JPEG,1
2,TIN/n01641577/images/n01641577_99.JPEG,2
3,TIN/n01644900/images/n01644900_99.JPEG,3
4,TIN/n01698640/images/n01698640_99.JPEG,4
...,...,...
195,TIN/n09246464/images/n09246464_99.JPEG,195
196,TIN/n09256479/images/n09256479_99.JPEG,196
197,TIN/n09332890/images/n09332890_99.JPEG,197
198,TIN/n09428293/images/n09428293_99.JPEG,198


In [5]:
train_img = []
for i in range(len(train["image"])):
    temp = cv2.imread(train["image"][i])
    temp = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)
    train_img.append(temp)

In [6]:
test_img = []
for i in range(len(test["image"])):
    temp = cv2.imread(test["image"][i])
    temp = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)
    test_img.append(temp)

In [7]:
# Extract image features
# SIFT
train_descriptors_sift = []
sift = cv2.xfeatures2d.SIFT_create()
for i in range(len(train_img)):
    keypoints, descriptors = sift.detectAndCompute(train_img[i], None)
    if descriptors is not None:
        train_descriptors_sift.extend(descriptors)
print(len(train_descriptors_sift))
train_descriptors_sift = np.array(train_descriptors_sift)

[ WARN:0@12.368] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv_contrib/modules/xfeatures2d/misc/python/shadow_sift.hpp (15) SIFT_create DEPRECATED: cv.xfeatures2d.SIFT_create() is deprecated due SIFT tranfer to the main repository. https://github.com/opencv/opencv/issues/16736


3654005


In [8]:
test_descriptors_sift = []
sift = cv2.xfeatures2d.SIFT_create()
for i in range(len(test_img)):
    keypoints, descriptors = sift.detectAndCompute(train_img[i], None)
    if descriptors is not None:
        test_descriptors_sift.extend(descriptors)
print(len(test_descriptors_sift))
test_descriptors_sift = np.array(test_descriptors_sift)

6071


In [9]:
# ORB
train_descriptors_orb = []
orb = cv2.ORB_create()
for i in range(len(train_img)):
    keypoints, descriptors = orb.detectAndCompute(train_img[i], None)
    if descriptors is not None:
        train_descriptors_orb.extend(descriptors)
print(len(train_descriptors_orb))
train_descriptors_orb = np.array(train_descriptors_orb)

21793


In [10]:
test_descriptors_orb = []
orb = cv2.ORB_create()
for i in range(len(test_img)):
    keypoints, descriptors = orb.detectAndCompute(test_img[i], None)
    if descriptors is not None:
        test_descriptors_orb.extend(descriptors)
print(len(test_descriptors_orb))
test_descriptors_orb = np.array(test_descriptors_orb)

44


In [11]:
#BRISK
train_descriptors_brisk = []
brisk = cv2.BRISK_create()
for i in range(len(train_img)):
    keypoints, descriptors = brisk.detectAndCompute(train_img[i], None)
    if descriptors is not None:
        train_descriptors_brisk.extend(descriptors)
print(len(train_descriptors_brisk))
train_descriptors_brisk = np.array(train_descriptors_brisk)

3165170


In [12]:
test_descriptors_brisk = []
brisk = cv2.BRISK_create()
for i in range(len(test_img)):
    keypoints, descriptors = brisk.detectAndCompute(test_img[i], None)
    if descriptors is not None:
        test_descriptors_brisk.extend(descriptors)
print(len(test_descriptors_brisk))
test_descriptors_brisk = np.array(test_descriptors_brisk)

6358


In [13]:
y_train = train["class"]
num_classes = len(y_train.unique())
print(num_classes)
y_test = test["class"]

200


In [14]:
from sklearn.cluster import KMeans
kmeans_sift_train = KMeans(n_clusters=40, random_state=42).fit(train_descriptors_sift)
kmeans_sift_test = KMeans(n_clusters=40, random_state=42).fit(test_descriptors_sift)
kmeans_orb_train = KMeans(n_clusters=40, random_state=42).fit(train_descriptors_orb)
kmeans_orb_test = KMeans(n_clusters=40, random_state=42).fit(test_descriptors_orb)
kmeans_brisk_train = KMeans(n_clusters=40, random_state=42).fit(train_descriptors_brisk)
kmeans_brisk_test = KMeans(n_clusters=40, random_state=42).fit(test_descriptors_brisk)

In [15]:
x_sift_train = []
y_sift_train = []
sift = cv2.xfeatures2d.SIFT_create()
for i in range(len(train_img)):
    keypoints, descriptors = sift.detectAndCompute(train_img[i], None)
    features = np.zeros(40)
    if descriptors is not None:
        labels = kmeans_sift_train.predict(descriptors)
        features = np.bincount(labels, minlength=40)
    x_sift_train.append(features)
    label = y_train[i]
    y_sift_train.append(int(label))
x_sift_train = np.asarray(x_sift_train)
y_sift_train = np.asarray(y_sift_train, np.int32)


In [16]:
x_sift_test = []
y_sift_test = []
sift = cv2.xfeatures2d.SIFT_create()
for i in range(len(test_img)):
    keypoints, descriptors = sift.detectAndCompute(test_img[i], None)
    features = np.zeros(40)
    if descriptors is not None:
        labels = kmeans_sift_test.predict(descriptors)
        features = np.bincount(labels, minlength=40)
    x_sift_test.append(features)
    label = y_test[i]
    y_sift_test.append(int(label))
x_sift_test = np.asarray(x_sift_test)
y_sift_test = np.asarray(y_sift_test, np.int32)

In [17]:
x_orb_train = []
y_orb_train = []
orb = cv2.ORB_create()
for i in range(len(train_img)):
    keypoints, descriptors = orb.detectAndCompute(train_img[i], None)
    features = np.zeros(40)
    if descriptors is not None:
        labels = kmeans_orb_train.predict(descriptors)
        features = np.bincount(labels, minlength=40)
    x_orb_train.append(features)
    label = y_train[i]
    y_orb_train.append(int(label))
x_orb_train = np.asarray(x_orb_train)
y_orb_train = np.asarray(y_orb_train, np.int32)

In [18]:
x_orb_test = []
y_orb_test = []
orb = cv2.ORB_create()
for i in range(len(test_img)):
    keypoints, descriptors = orb.detectAndCompute(test_img[i], None)
    features = np.zeros(40)
    if descriptors is not None:
        labels = kmeans_orb_test.predict(descriptors)
        features = np.bincount(labels, minlength=40)
    x_orb_test.append(features)
    label = y_test[i]
    y_orb_test.append(int(label))
x_orb_test = np.asarray(x_orb_test)
y_orb_test = np.asarray(y_orb_test, np.int32)

In [19]:
x_brisk_train = []
y_brisk_train = []
brisk = cv2.BRISK_create()
for i in range(len(train_img)):
    keypoints, descriptors = brisk.detectAndCompute(train_img[i], None)
    features = np.zeros(40)
    if descriptors is not None:
        labels = kmeans_brisk_train.predict(descriptors)
        features = np.bincount(labels, minlength=40)
    x_brisk_train.append(features)
    label = y_train[i]
    y_brisk_train.append(int(label))
x_brisk_train = np.asarray(x_brisk_train)
y_brisk_train = np.asarray(y_brisk_train, np.int32)

In [20]:
x_brisk_test = []
y_brisk_test = []
brisk = cv2.BRISK_create()
for i in range(len(test_img)):
    keypoints, descriptors = brisk.detectAndCompute(test_img[i], None)
    features = np.zeros(40)
    if descriptors is not None:
        labels = kmeans_brisk_test.predict(descriptors)
        features = np.bincount(labels, minlength=40)
    x_brisk_test.append(features)
    label = y_test[i]
    y_brisk_test.append(int(label))
x_brisk_test = np.asarray(x_brisk_test)
y_brisk_test = np.asarray(y_brisk_test, np.int32)

In [21]:
# KNN # SIFT
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(x_sift_train, y_sift_train)
y_knn_pred = knn.predict(x_sift_test)

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

accuracy = accuracy_score(y_sift_test, y_knn_pred)
f1 = f1_score(y_sift_test, y_knn_pred, average = "macro")
print("SIFT: Accuracy - ", accuracy, "F1 Score - ", f1)

SIFT: Accuracy -  0.005 F1 Score -  0.0016666666666666668


In [22]:
# KNN # ORB
knn = KNeighborsClassifier()
knn.fit(x_orb_train, y_orb_train)
y_knn_pred = knn.predict(x_orb_test)

accuracy = accuracy_score(y_orb_test, y_knn_pred)
f1 = f1_score(y_orb_test, y_knn_pred, average = "macro")
print("ORB: Accuracy - ", accuracy, "F1 Score - ", f1)

ORB: Accuracy -  0.005 F1 Score -  5.8479532163742686e-05


In [23]:
# KNN # BRISK
knn = KNeighborsClassifier()
knn.fit(x_brisk_train, y_brisk_train)
y_knn_pred = knn.predict(x_brisk_test)

accuracy = accuracy_score(y_brisk_test, y_knn_pred)
f1 = f1_score(y_brisk_test, y_knn_pred, average = "macro")
print("BRISK: Accuracy - ", accuracy, "F1 Score - ", f1)

BRISK: Accuracy -  0.01 F1 Score -  0.0034285714285714284


In [24]:
# SVM # SIFT
from sklearn.svm import SVC
svm = SVC()
svm.fit(x_sift_train, y_sift_train)
y_svm_pred = svm.predict(x_sift_test)

accuracy = accuracy_score(y_sift_test, y_svm_pred)
f1 = f1_score(y_sift_test, y_svm_pred, average = "macro")
print("SIFT: Accuracy - ", accuracy, "F1 Score - ", f1)

SIFT: Accuracy -  0.02 F1 Score -  0.01092857142857143


In [25]:
# SVM # ORB
svm = SVC()
svm.fit(x_orb_train, y_orb_train)
y_svm_pred = svm.predict(x_orb_test)

accuracy = accuracy_score(y_orb_test, y_svm_pred)
f1 = f1_score(y_orb_test, y_svm_pred, average = "macro")
print("ORB: Accuracy - ", accuracy, "F1 Score - ", f1)

ORB: Accuracy -  0.005 F1 Score -  6.369426751592357e-05


In [26]:
# SVM # BRISK
svm = SVC()
svm.fit(x_brisk_train, y_brisk_train)
y_svm_pred = svm.predict(x_brisk_test)

accuracy = accuracy_score(y_brisk_test, y_svm_pred)
f1 = f1_score(y_brisk_test, y_svm_pred, average = "macro")
print("BRISK: Accuracy - ", accuracy, "F1 Score - ", f1)

BRISK: Accuracy -  0.005 F1 Score -  0.00125


In [27]:
# My model
# Catboost
!pip install catboost

  Using cached catboost-1.2.3.tar.gz (69.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached graphviz-0.20.1-py3-none-any.whl.metadata (12 kB)
Using cached graphviz-0.20.1-py3-none-any.whl (47 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for catboost (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [98 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-10.9-x86_64-cpython-37
      creating build/lib.macosx-10.9-x86_64-cpython-37/catboost
      copying catboost/monoforest.py -> build/lib.macosx-10.9-x86_64-cpython-37/catboost
      copying catboost/plot_helpers.py -> build/lib.macosx-10.9-x86_64-cpython-37/catboost
      copying catboost/metrics.py -> build/lib.macosx-10.9-x86_64-cpython-37/catboost
      copying catboost/carry.py -> build/lib.macosx-10.9-x86_64-

In [28]:
# Catboost # SIFT
from catboost import CatBoostClassifier
catboost = CatBoostClassifier()
catboost.fit(x_sift_train, y_sift_train)
y_catboost_pred = catboost.predict(x_sift_test)

accuracy = accuracy_score(y_sift_test, y_catboost_pred)
f1 = f1_score(y_sift_test, y_catboost_pred, average = "macro")
print("SIFT: Accuracy - ", accuracy, "F1 Score - ", f1)

Learning rate set to 0.100054
0:	learn: 5.2757043	total: 2.07s	remaining: 34m 30s
1:	learn: 5.2573970	total: 3.88s	remaining: 32m 14s
2:	learn: 5.2370458	total: 5.47s	remaining: 30m 17s
3:	learn: 5.2203470	total: 7.25s	remaining: 30m 5s
4:	learn: 5.2014354	total: 8.88s	remaining: 29m 27s
5:	learn: 5.1823723	total: 10.5s	remaining: 28m 52s
6:	learn: 5.1651563	total: 12s	remaining: 28m 18s
7:	learn: 5.1518388	total: 14.4s	remaining: 29m 46s
8:	learn: 5.1392222	total: 16.2s	remaining: 29m 45s
9:	learn: 5.1208450	total: 17.8s	remaining: 29m 22s
10:	learn: 5.1047137	total: 19.4s	remaining: 29m 4s
11:	learn: 5.0886858	total: 20.9s	remaining: 28m 39s
12:	learn: 5.0744774	total: 22.5s	remaining: 28m 28s
13:	learn: 5.0609492	total: 24.1s	remaining: 28m 17s
14:	learn: 5.0486071	total: 25.7s	remaining: 28m 5s
15:	learn: 5.0371863	total: 27.3s	remaining: 27m 56s
16:	learn: 5.0295598	total: 29s	remaining: 27m 58s
17:	learn: 5.0197658	total: 30.6s	remaining: 27m 47s
18:	learn: 5.0101554	total: 32.3s

In [29]:
# Catboost # ORB
from catboost import CatBoostClassifier
catboost = CatBoostClassifier()
catboost.fit(x_orb_train, y_orb_train)
y_catboost_pred = catboost.predict(x_orb_test)

accuracy = accuracy_score(y_orb_test, y_catboost_pred)
f1 = f1_score(y_orb_test, y_catboost_pred, average = "macro")
print("ORB: Accuracy - ", accuracy, "F1 Score - ", f1)

Learning rate set to 0.100054
0:	learn: 5.2976059	total: 971ms	remaining: 16m 9s
1:	learn: 5.2969523	total: 1.86s	remaining: 15m 28s
2:	learn: 5.2962894	total: 2.76s	remaining: 15m 16s
3:	learn: 5.2955785	total: 3.63s	remaining: 15m 4s
4:	learn: 5.2949328	total: 4.54s	remaining: 15m 4s
5:	learn: 5.2942808	total: 5.41s	remaining: 14m 56s
6:	learn: 5.2934659	total: 6.28s	remaining: 14m 51s
7:	learn: 5.2927927	total: 7.15s	remaining: 14m 46s
8:	learn: 5.2921762	total: 8.09s	remaining: 14m 50s
9:	learn: 5.2916398	total: 8.97s	remaining: 14m 48s
10:	learn: 5.2909458	total: 9.83s	remaining: 14m 44s
11:	learn: 5.2904039	total: 10.7s	remaining: 14m 43s
12:	learn: 5.2898981	total: 11.6s	remaining: 14m 40s
13:	learn: 5.2892749	total: 12.5s	remaining: 14m 38s
14:	learn: 5.2886907	total: 13.4s	remaining: 14m 36s
15:	learn: 5.2881985	total: 14.2s	remaining: 14m 33s
16:	learn: 5.2876261	total: 15.1s	remaining: 14m 31s
17:	learn: 5.2870379	total: 15.9s	remaining: 14m 29s
18:	learn: 5.2868978	total: 1

In [30]:
# Catboost # BRISK
from catboost import CatBoostClassifier
catboost = CatBoostClassifier()
catboost.fit(x_brisk_train, y_brisk_train)
y_catboost_pred = catboost.predict(x_brisk_test)

accuracy = accuracy_score(y_brisk_test, y_catboost_pred)
f1 = f1_score(y_brisk_test, y_catboost_pred, average = "macro")
print("BRISK: Accuracy - ", accuracy, "F1 Score - ", f1)

Learning rate set to 0.100054
0:	learn: 5.2782747	total: 1.8s	remaining: 30m 1s
1:	learn: 5.2604656	total: 3.56s	remaining: 29m 34s
2:	learn: 5.2415628	total: 5.21s	remaining: 28m 52s
3:	learn: 5.2224913	total: 6.92s	remaining: 28m 43s
4:	learn: 5.2074619	total: 8.7s	remaining: 28m 51s
5:	learn: 5.1949390	total: 10.6s	remaining: 29m 13s
6:	learn: 5.1817886	total: 12.2s	remaining: 28m 55s
7:	learn: 5.1736242	total: 14.2s	remaining: 29m 23s
8:	learn: 5.1627616	total: 15.9s	remaining: 29m 13s
9:	learn: 5.1488401	total: 17.7s	remaining: 29m 10s
10:	learn: 5.1356772	total: 19.2s	remaining: 28m 47s
11:	learn: 5.1245481	total: 21s	remaining: 28m 50s
12:	learn: 5.1128313	total: 22.9s	remaining: 28m 58s
13:	learn: 5.1045619	total: 24.9s	remaining: 29m 17s
14:	learn: 5.0943237	total: 26.7s	remaining: 29m 13s
15:	learn: 5.0863763	total: 28.5s	remaining: 29m 11s
16:	learn: 5.0804040	total: 30.5s	remaining: 29m 21s
17:	learn: 5.0733774	total: 32.2s	remaining: 29m 17s
18:	learn: 5.0650591	total: 34s

In [31]:
# Tree # SIFT
from sklearn.tree import DecisionTreeClassifier
tree=DecisionTreeClassifier(criterion="entropy", random_state=42)
tree.fit(x_sift_train, y_sift_train)
y_tree_pred=tree.predict(x_sift_test)

accuracy = accuracy_score(y_sift_test, y_tree_pred)
f1 = f1_score(y_sift_test, y_tree_pred, average = "macro")
print("SIFT: Accuracy - ", accuracy, "F1 Score - ", f1)

SIFT: Accuracy -  0.01 F1 Score -  0.005333333333333333


In [32]:
# Tree # ORB
from sklearn.tree import DecisionTreeClassifier
tree=DecisionTreeClassifier(criterion="entropy", random_state=42)
tree.fit(x_orb_train, y_orb_train)
y_tree_pred=tree.predict(x_orb_test)

accuracy = accuracy_score(y_orb_test, y_tree_pred)
f1 = f1_score(y_orb_test, y_tree_pred, average = "macro")
print("SIFT: Accuracy - ", accuracy, "F1 Score - ", f1)

SIFT: Accuracy -  0.005 F1 Score -  6.369426751592357e-05


In [33]:
# Tree # BRISK
from sklearn.tree import DecisionTreeClassifier
tree=DecisionTreeClassifier(criterion="entropy", random_state=42)
tree.fit(x_brisk_train, y_brisk_train)
y_tree_pred=tree.predict(x_brisk_test)

accuracy = accuracy_score(y_brisk_test, y_tree_pred)
f1 = f1_score(y_brisk_test, y_tree_pred, average = "macro")
print("SIFT: Accuracy - ", accuracy, "F1 Score - ", f1)

SIFT: Accuracy -  0.0 F1 Score -  0.0
